In [11]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import numpy as np
import pandas as pd
from pathlib import Path
import lightgbm as lgb
import datetime as dt
import optuna
import gc
import warnings
import random
import os
from functools import partial
from scipy import stats
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

warnings.simplefilter('ignore')

In [13]:
def fix_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    
fix_all_seeds(0)

# データ読み込み

In [14]:
df_train_val = pd.read_pickle("/content/drive/MyDrive/Kaggle/BlueCarbon/proc/Val_20230429/train_predicted_val10-1.pkl")
df_train_val.head()

,MINMAX_SWIR1__mean,MINMAX_BWDRVI__mean,MINMAXDiff_ExG,MINMAX_mCRIRE__mean,MINMAXDiff_BGI,MED_NIR_median,RDVI,MINMAXDiff_B07,MINMAXDiff_TCI,MINMAX_D678_500__mean,...,NDMI,MINMAX_GVMI__std,MED_CVI_std,aicial_length,MED_TIRS1_median,MINMAX_VARIgreen,MINMAX_Cirededge__std,MINMAXDiff_RGR,MED_B06,y_pred
0,0.227549,0.090928,0.033562,-0.342965,0.756266,-0.493991,-1.902453,256.030609,27.275394,-0.084711,...,0.884876,1.066376,0.906956,0.000000,0.444899,0.734551,1.099084,0.477341,128.356201,0.051648
1,-1.138076,-0.486539,0.032487,1.319135,0.350933,0.702148,1.756928,2529.655762,89.906555,-0.357982,...,0.833784,0.993003,1.940318,0.478772,0.079011,2.862382,2.048537,0.160493,1480.778442,0.634447
2,0.328841,0.114654,0.018530,0.092055,0.184313,-0.150096,-2.078368,440.799927,71.606461,-0.188156,...,0.848540,1.338313,1.282929,180.985031,-0.216197,-1.081439,1.289417,0.064769,1322.071411,0.051965
3,-0.540155,-0.885264,0.028959,-0.772320,0.360510,-0.671032,NaN,878.505310,69.952789,-0.413223,...,NaN,0.747443,0.224265,0.361444,-0.091160,NaN,0.404593,0.347347,442.730255,0.629431
4,-0.034661,-0.147565,0.022152,-0.530615,0.164914,-0.534631,-1.435234,707.599792,63.780663,0.330788,...,0.909053,0.712074,0.469220,6.062498,-0.188908,0.069282,0.651454,0.367770,416.059448,0.187677


# 特徴量

In [15]:
features = [
    'CTVI',
    'MINMAX_Cigreen',
    'MINMAXDiff_ExG',
    'NIR',
    'MINMAX_mCRIRE__mean',
    'MED_Datt1',
    'fetch',
    'MED_CTVI_median',
    'SLAVI',
    'MINMAXDiff_BGI',
    'MED_NIR_median',
    'MINMAX_TIRS1',
    'TIRS1',
    'RDVI',
    'MINMAX_TIRS1__std',
    'MED_ARI',
    'MINMAXDiff_TCI',
    'MINMAX_D678_500__mean',
    'MINMAXDiff_GDVI',
    'MINMAX_NLI__mean',
    'MINMAXDiff_NDVI2',
    'MINMAX_DSWI_5__std',
    'PPR',
    'MED_IR550_std',
    'MINMAXDiff_BWDRVI',
    'MED_SIPI2_median',
    'area',
    'MED_SWIR2_median',
    'MED_MSBI',
    'MINMAX_Cirededge__std',
    'MINMAX_NormG__std',
    'MED_AVI',
    'MINMAX_TIRS1__mean',
    'MINMAX_Alteration',
    'MINMAX_GARI__std',
    'MED_NDRE',
    'MED_D678_500_median',
    'MED_GLI_std',
    'MVI',
    'MINMAX_TSAVI',
    'MINMAX_Fe3',
    'MINMAX_SIPI2__mean',
    'NLI',
    'MINMAX_Gossan',
    'cliff_length',
    'MED_MYVI',
    'MINMAX_Green',
    'MINMAXDiff_Chlgreen',
    'MINMAX_Ferrous_Silicates__std',
    'MINMAX_NIR__std',
    'MINMAX_CRI550',
    'MINMAX_Blue',
    'MINMAX_D678_500__std',
    'MINMAXDiff_Rededge1',
    'SWIR1',
    'MINMAX_IR550',
    'MED_mCRIRE_std',
    'SIPI2',
    'MED_Fe3_median',
    'MINMAX_SLAVI__mean',
    'MED_PPR_median',
    'MED_BOA',
    'MINMAX_DSWI',
    'MINMAX_SWIR2',
    'MINMAX_SLAVI__std',
    'NormG',
    'cold_sst',
    'MED_RGR',
    'MINMAX_Green__std',
    'MED_NormR_median',
    'MINMAXDiff_B05',
    'MED_Green_std',
    'MINMAX_CTVI__mean',
    'MINMAX_DVIMSS',
    'MINMAX_TSAVI__mean',
    'DSWI',
    'MINMAXDiff_MSR670',
    'MED_AFRI1600',
    'MINMAX_IF',
    'MINMAX_CI__std',
    'VARIgreen',
    'MED_RDVI_std',
    'MINMAX_CI',
    'MED_NormR_std',
    'MINMAX_Chlred_edge__mean',
    'MINMAXDiff_CIgreen',
    'MED_SLAVI_std',
    'MED_Ferric_Oxides_median',
    'MINMAXDiff_GARI',
    'MED_Blue_std',
    'MINMAX_Red',
    'MINMAX_VARIgreen__std',
    'MINMAX_MCARI2',
    'MED_GLI_median',
    'MED_MCARI_OSAVI',
    'MED_mCRIG_median',
    'MINMAXDiff_Datt4',
    'MINMAXDiff_CRred',
    'MINMAX_MVI__mean',
    'MED_GEMI_median',
    'MED_EVI_median',
    'MED_GARI_std',
    'river_area',
    'MED_NDFI2',
    'MINMAX_EVI__std',
    'Fe3',
    'MINMAX_Ferric_Oxides__std',
    'MED_Blue_median',
    'MINMAX_MCARI1__std',
    'MINMAXDiff_B06',
    'sunAzimuth',
    'MINMAX_NormR__mean',
    'MED_CIgreen',
    'Green',
    'MED_CRI550_std',
    'MED_CI_median',
    'MINMAXDiff_Datt1',
    'MED_mCRIRE_median',
    'MED_Gossan_median',
    'MINMAXDiff_GI',
    'mCRIRE',
    'Red',
    'MED_CRI550_median',
    'MINMAXDiff_MSBI',
    'MED_TSAVI_std',
    'MINMAX_NLI__std',
    'MED_Chlred_edge',
    'H',
    'MINMAX_MCARI1',
    'MINMAXDiff_DVI',
    'MED_BGI',
    'MINMAXDiff_NDRE',
    'MINMAXDiff_Chlred_edge',
    'MED_CIrededge',
    'depth_original',
    'MINMAX_NLI',
    'MINMAXDiff_AFRI1600',
    'MED_GLI',
    'MINMAXDiff_B03',
    'MINMAX_SLAVI',
    'MINMAX_H__mean',
    'IF',
    'MINMAX_Chlgreen',
    'MED_WVP',
    'MINMAXDiff_IF',
    'MINMAX_VARIgreen__mean',
    'MINMAX_GLI',
    'MINMAX_Fe3__mean',
    'MINMAX_GEMI',
    'MINMAXDiff_MCARI_OSAVI',
    'MINMAX_mCRIRE__std',
    'Landsat_num',
    'MED_mNDVI',
    'MINMAXDiff_NDWI2',
    'MINMAXDiff_SR672_708',
    'MINMAX_NormR',
    'MED_MCARI2',
    'MED_MCARI_MTVI2',
    'MED_SWIR2_std',
    'MINMAX_PPR',
    'MED_MNSI',
    'MED_GVMI_median',
    'MED_TIRS2_median',
    'MED_B11',
    'MINMAX_GEMI__std',
    'MINMAX_Red__mean',
    'coastal_dist',
    'Alteration',
    'MED_NDVI2',
    'GARI',
    'MED_MVI',
    'MINMAX_NIR',
    'MINMAXDiff_MIRBI',
    'MINMAX_Ferric_Oxides',
    'warm_sst',
    'MINMAX_MCARI2__std',
    'MED_MSR670',
    'MED_CSI',
    'MINMAX_Chlgreen__mean',
    'MED_CLD',
    'MINMAX_EVI__mean',
    'MINMAX_MCARI1__mean',
    'MINMAX_Green__mean',
    'MINMAX_EVI',
    'MINMAXDiff_ARI',
    'MED_Rededge1',
    'MINMAX_DVIMSS__std',
    'MINMAX_CTVI',
    'MINMAX_NormG',
    'sst_ymd',
    'MED_DSWI_std',
    'MINMAX_RDVI__std',
    'MINMAX_RDVI__mean',
    'MINMAX_mCRIG__mean',
    'NBR2',
    'MINMAX_IR550__mean',
    'MINMAXDiff_BRI',
    'MED_NormG_std',
    'MINMAX_mCRIG__std',
    'MINMAX_TIRS2',
    'MINMAX_H__std',
    'GEMI',
    'MED_PSSRc2',
    'MED_MCARI1_median',
    'MED_SWIR1_std',
    'MINMAX_SIPI2__std',
    'MINMAXDiff_NDFI2',
    'MINMAX_NormG__mean',
    'MED_SLAVI_median',
    'Chlred_edge',
    'EVI',
    'MED_NDWI2',
    'MINMAXDiff_B02',
    'MINMAX_RDVI',
    'MINMAX_Blue__mean',
    'MED_TSAVI_median',
    'MINMAXDiff_IO',
    'MED_BNDVI',
    'MED_CTVI_std',
    'MED_Red_median',
    'MINMAXDiff_MVI',
    'MED_D678_500_std',
    'Ferrous_Silicates',
    'MINMAX_GVMI__mean',
    'MED_RDVI_median',
    'MINMAXDiff_NBR2',
    'MINMAX_CRI550__std',
    'MINMAX_IR550__std',
    'MINMAX_Ferrous_Silicates__mean',
    'MED_DSWI_5_std',
    'MINMAX_Gossan__mean',
    'MED_TIRS1_std',
    'MINMAX_SWIR2__std',
    'MINMAX_BWDRVI',
    'MINMAXDiff_WVP',
    'MED_Cirededge_median',
    'MINMAXDiff_EVI',
    'D678_500',
    'MINMAX_SWIR1',
    'MINMAXDiff_D678_500',
    'MED_TIRS2_std',
    'MINMAX_CVI__mean',
    'MINMAXDiff_MGVI',
    'MED_Gossan_std',
    'GVMI',
    'MINMAXDiff_CARI',
    'MED_MCARI2_std',
    'MED_AVI_median',
    'MED_VARIgreen_std',
    'MINMAX_SIPI2',
    'MINMAX_Blue__std',
    'SAVI',
    'MINMAX_Fe3__std',
    'MED_NDVIre',
    'MINMAX_PPR__mean',
    'MED_MVI_median',
    'MINMAX_Chlgreen__std',
    'MINMAXDiff_mSR',
    'MED_GDVI',
    'MED_DVIMSS_std',
    'MED_MIRBI',
    'MINMAX_Alteration__std',
    'MINMAX_Chlred_edge',
    'MINMAXDiff_CLD',
    'MED_BWDRVI_median',
    'MED_MSI',
    'CRI550',
    'MED_Cigreen_median',
    'MED_EVI_std',
    'MINMAXDiff_GRNDVI',
    'GLI',
    'MED_Red_std',
    'MINMAXDiff_mNDVI',
    'MED_Ferrous_Silicates_std',
    'MED_CARI',
    'MINMAXDiff_PSSRc2',
    'MED_NDSI',
    'MED_Bcc',
    'MINMAX_CTVI__std',
    'MED_CVI',
    'MED_VARIgreen_median',
    'MINMAX_Ferric_Oxides__mean',
    'NDWI',
    'MINMAXDiff_B12',
    'MED_NIR_std',
    'MED_NBR2',
    'MED_NLI_median',
    'MED_CRred',
    'Blue',
    'MED_GARI',
    'sunElevation',
    'MED_B8A',
    'MINMAX_SWIR2__mean',
    'MINMAX_MCARI2__mean',
    'river_dist',
    'MED_GVMI_std',
    'MINMAX_Red__std',
    'hist_warm_sst',
    'MED_GVMI',
    'MINMAX_IF__mean',
    'MINMAX_Ferrous_Silicates',
    'MINMAX_AVI',
    'IR550',
    'MINMAX_MVI',
    'SWIR2',
    'MINMAX_MVI__std',
    'MED_Green_median',
    'MED_Ferrous_Silicates_median',
    'MINMAX_GEMI__mean',
    'MED_GI',
    'MED_MCARI2_median',
    'MINMAX_DVIMSS__mean',
    'MINMAX_D678_500',
    'MINMAX_DSWI_5__mean',
    'MINMAX_H',
    'MED_TCI',
    'MED_IR550_median',
    'MED_SB550',
    'AVI',
    'MINMAX_mCRIRE',
    'MED_NDII',
    'MED_H_std',
    'MINMAXDiff_ARVI',
    'MINMAX_CVI__std',
    'MED_ARVI',
    'MINMAX_TSAVI__std',
    'Cirededge',
    'MINMAXDiff_AOT',
    'MED_BWDRVI',
    'MED_MGVI',
    'Gossan',
    'MINMAXDiff_MYVI',
    'MED_Chlred_edge_median',
    'Cigreen',
    'MINMAX_NIR__mean',
    'MINMAX_GARI__mean',
    'MED_SWIR1_median',
    'MINMAXDiff_GVMI',
    'MINMAX_Gossan__std',
    'MED_GEMI_std',
    'Salinity_annual',
    'MED_D678_500',
    'MINMAX_CRI550__mean',
    'MINMAX_TIRS2__std',
    'MED_H_median',
    'MINMAX_Cigreen__std',
    'MINMAXDiff_NDSI',
    'MED_B02',
    'BWDRVI',
    'MINMAX_DSWI_5',
    'MINMAXDiff_GLI',
    'mCRIG',
    'MINMAX_PPR__std',
    'year',
    'MINMAXDiff_CSI',
    'MED_mSR',
    'MINMAXDiff_MCARI2',
    'MED_NormG_median',
    'MINMAX_BWDRVI__std',
    'MINMAX_GLI__std',
    'MINMAX_GARI',
    'MED_PPR_std',
    'MINMAX_DSWI__std',
    'sst_diff',
    'MED_B12',
    'MINMAX_GVMI',
    'MINMAX_SWIR1__std',
    'MED_BRI',
    'MINMAX_Alteration__mean',
    'MED_mCRIG_std',
    'MINMAXDiff_BB1',
    'MINMAXDiff_MCARI_MTVI2',
    'MCARI2',
    'MED_MCARI1_std',
    'MED_IO',
    'MED_DVIMSS_median',
    'MED_Chlgreen_std',
    'MINMAX_Chlred_edge__std',
    'TIRS2',
    'MED_AOT',
    'MED_DVI',
    'MED_DSWI_median',
    'MED_Fe3_std',
    'MINMAXDiff_MNSI',
    'MED_IF_median',
    'MED_Alteration_median',
    'MINMAX_IF__std',
    'MCARI1',
    'coast_length',
    'MINMAXDiff_AVI',
    'MINMAX_GLI__mean',
    'depth',
    'MINMAX_TIRS2__mean',
    'MED_BWDRVI_std',
    'beach_length',
    'MED_Alteration_std',
    'MINMAXDiff_NDVIre',
    'MED_SR672_708',
    'MINMAX_AVI__std',
    'MINMAXDiff_B11',
    'Chlgreen',
    'MINMAX_AVI__mean',
    'MINMAXDiff_BOA',
    'MINMAX_NormR__std',
    'hist_cold_sst',
    'MINMAX_mCRIG',
    'MED_CI_std',
    'MINMAX_PSNDc2__mean',
    'MED_MVI_std',
    'MINMAXDiff_CIrededge',
    'MED_AVI_std',
    'MED_Ferric_Oxides_std',
    'MED_GARI_median',
    'MINMAX_CVI',
    'MED_IF',
    'MINMAXDiff_Bcc',
    'MED_SIPI2_std',
    'MINMAX_Cigreen__mean',
    'MED_Datt4',
    'MED_EVI',
    'MED_ExG',
    'NormR',
    'DVIMSS',
    'MINMAX_CI__mean',
    'NDMI',
    'MINMAX_GVMI__std',
    'MINMAXDiff_RDI',
    'MED_IF_std',
    'MED_CVI_std',
    'aicial_length',
    'MED_Chlgreen_median',
    'MED_TIRS1_median',
    'MED_Chlred_edge_std',
    'MED_NLI_std',
    'MINMAX_VARIgreen',
    'MINMAXDiff_RGR',
    'MED_Cirededge_std',
    'CI'
]

print("特徴量の数：", len(features))

特徴量の数： 438


# Optuna

In [16]:
def my_kfold(df):
    # idx = df.index
    # cv = []
    # train_idx1 = (idx < dt.datetime(2018, 4, 1))
    # val_idx1 = (idx >= dt.datetime(2018, 4, 1))&\
    #            (idx < dt.datetime(2018, 4, 15))
    # cv.append([train_idx1, val_idx1])
    # train_idx2 = (idx < dt.datetime(2018, 4, 15))
    # val_idx2 = (idx >= dt.datetime(2018, 4, 15))&\
    #            (idx < dt.datetime(2018, 4, 29))
    # cv.append([train_idx2, val_idx2])
    # train_idx3 = (idx < dt.datetime(2019, 4, 1))
    # val_idx3 = (idx >= dt.datetime(2019, 4, 1))&\
    #            (idx < dt.datetime(2019, 4, 15))
    # cv.append([train_idx3, val_idx3])
    # train_idx4 = (idx < dt.datetime(2019, 4, 15))
    # val_idx4 = (idx >= dt.datetime(2019, 4, 15))&\
    #            (idx < dt.datetime(2019, 4, 29))
    # cv.append([train_idx4, val_idx4])

    kfold = KFold(n_splits=3, shuffle=True, random_state=42)
    cv = kfold.split(df)
    return cv

In [17]:
def objective(X, y, y_raw, trial):

    # Optuna suggest params
    params = {
        # "extra_trees": True,
        'objective': 'fair',
        'metric': 'fair',
        'boosting_type': 'gbdt',
        'max_depth': trial.suggest_int('max_depth', 3, 21),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.002, 0.10),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 0.05, 10.0),
        # 'lambda_l2': trial.suggest_loguniform('lambda_l2', 0.05, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 5, 256),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3, 0.8),
        # 'subsample': trial.suggest_uniform('subsample', 0.4, 0.9),
#         'subsample_freq': trial.suggest_int('subsample_freq', 1, 7),
#         'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        "verbose": -1,
        'seed': 46,
        "n_jobs": -1
    }

    # FN_ratio = trial.suggest_uniform('FN_ratio', 0, 1)

    kfold = KFold(n_splits=3, shuffle=True, random_state=42)
    
    scores = []
    for train_idx, val_idx in kfold.split(X):
        X_train = X[train_idx]
        y_train = y[train_idx]
        X_val = X[val_idx]
        y_val = y[val_idx]
        y_val_raw = y_raw[val_idx]
       
        train_dataset = lgb.Dataset(X_train, y_train, feature_name=features)
        
        model = lgb.train(params, train_dataset)
        y_pred = model.predict(X_val)
        y_pred = np.expm1(y_pred)
        y_pred[y_pred<0] = 0
        y_pred[y_pred>1] = 1

        rmse = np.sqrt(mean_squared_error(y_val_raw, y_pred))
        scores.append(rmse)
    print("Scores:", scores)
    return np.average(scores)

In [18]:
# cv_indicies = my_kfold(df_train_val)

train_val_X = df_train_val[features].values
train_val_y = df_train_val["log_cover"].values
train_val_y_raw = df_train_val["cover"].values

# del df_train_val
# gc.collect()

In [19]:
f = partial(objective,
            train_val_X,
            train_val_y,
            train_val_y_raw)
study = optuna.create_study(direction="minimize")
study.optimize(f, n_trials=100)

trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
best_params = trial.params  

[I 2023-04-29 15:17:14,552] A new study created in memory with name: no-name-2196f7da-fff5-4231-8abf-25277b14e077
[I 2023-04-29 15:20:25,886] Trial 0 finished with value: 0.11939624646786473 and parameters: {'max_depth': 10, 'n_estimators': 2545, 'learning_rate': 0.02483012645827288, 'lambda_l1': 2.6996334475333796, 'num_leaves': 196, 'colsample_bytree': 0.5407068922070917}. Best is trial 0 with value: 0.11939624646786473.


Scores: [0.11960558120475943, 0.11940058257249005, 0.11918257562634468]


[I 2023-04-29 15:21:40,041] Trial 1 finished with value: 0.12828495617730468 and parameters: {'max_depth': 9, 'n_estimators': 2271, 'learning_rate': 0.05773735241411397, 'lambda_l1': 3.8424392612173635, 'num_leaves': 10, 'colsample_bytree': 0.42498149650781525}. Best is trial 0 with value: 0.11939624646786473.


Scores: [0.12839367298344836, 0.12762506431919476, 0.1288361312292709]


[I 2023-04-29 15:30:45,995] Trial 2 finished with value: 0.11416242592120833 and parameters: {'max_depth': 17, 'n_estimators': 2925, 'learning_rate': 0.08061920926109595, 'lambda_l1': 0.07178350385695681, 'num_leaves': 180, 'colsample_bytree': 0.7401658481398823}. Best is trial 2 with value: 0.11416242592120833.


...

Scores: [0.11553998499771082, 0.11421765955108476, 0.11006770229128758]


[I 2023-04-29 22:36:21,230] Trial 35 finished with value: 0.11264784171702637 and parameters: {'max_depth': 17, 'n_estimators': 2437, 'learning_rate': 0.017249570940885327, 'lambda_l1': 0.07458308290764872, 'num_leaves': 170, 'colsample_bytree': 0.6731079655436575}. Best is trial 35 with value: 0.11264784171702637.


Scores: [0.11423865689603456, 0.11395738744370618, 0.10974748081133841]


[W 2023-04-29 23:22:30,285] Trial 36 failed with parameters: {'max_depth': 17, 'n_estimators': 2551, 'learning_rate': 0.009200840899621206, 'lambda_l1': 0.06579989439359314, 'num_leaves': 195, 'colsample_bytree': 0.7783042440658643} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-17-05d820403dd9>", line 38, in objective
    model = lgb.train(params, train_dataset)
  File "/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py", line 292, in train
    booster.update(fobj=fobj)
  File "/usr/local/lib/python3.10/dist-packages/lightgbm/basic.py", line 3021, in update
    _safe_call(_LIB.LGBM_BoosterUpdateOneIter(
KeyboardInterrupt
[W 2023-04-29 23:22:30,299] Trial 36 failed with value None.


KeyboardInterrupt: ignored

In [ ]:
best_params